In [1]:
conda install -c conda-forge folium

Retrieving notices: ...working... done
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import json
import os
from datetime import datetime
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 50)
from tqdm import tqdm

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import glob

df_dict =dict()

files = glob.glob("/Users/linahrusere/Documents/EHR/csv 2/*.csv")
for a in files:
    df_name = a.split('/')[-1].split('.')[0] + "_df"
    df= pd.read_csv(a)
    df_dict[df_name] = df


In [5]:
df_dict.keys()

dict_keys(['medications_df', 'providers_df', 'payer_transitions_df', 'imaging_studies_df', 'supplies_df', 'payers_df', 'allergies_df', 'procedures_df', 'organizations_df', 'conditions_df', 'careplans_df', 'encounters_df', 'devices_df', 'immunizations_df', 'patients_df', 'observations_df'])

# Explore demographic data 

In [6]:
patients_df = df_dict['patients_df']
patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   1171 non-null   object 
 1   BIRTHDATE            1171 non-null   object 
 2   DEATHDATE            171 non-null    object 
 3   SSN                  1171 non-null   object 
 4   DRIVERS              958 non-null    object 
 5   PASSPORT             898 non-null    object 
 6   PREFIX               927 non-null    object 
 7   FIRST                1171 non-null   object 
 8   LAST                 1171 non-null   object 
 9   SUFFIX               12 non-null     object 
 10  MAIDEN               331 non-null    object 
 11  MARITAL              791 non-null    object 
 12  RACE                 1171 non-null   object 
 13  ETHNICITY            1171 non-null   object 
 14  GENDER               1171 non-null   object 
 15  BIRTHPLACE           1171 non-null   o

In [7]:
patients_df['BIRTHDATE'] =pd.to_datetime(patients_df['BIRTHDATE'], infer_datetime_format=True)
patients_df['DEATHDATE'] =pd.to_datetime(patients_df['DEATHDATE'], infer_datetime_format=True)
#Calculate age
patients_df["finalyear"]= patients_df["DEATHDATE"].apply(lambda x : (x.year)).fillna(2020)
patients_df["firstyear"]= patients_df["BIRTHDATE"].apply(lambda x : (x.year))
patients_df["AGE"] = patients_df["finalyear"] - patients_df["firstyear"]
patients_df=patients_df.drop(columns=["finalyear","firstyear"])

### Careplans

In [8]:
careplans_df = df_dict['careplans_df']
list(careplans_df['DESCRIPTION'].unique())

['Respiratory therapy',
 'Wound care',
 'Physical therapy procedure',
 'Urinary tract infection care',
 'Fracture care',
 'Routine antenatal care',
 'Diabetes self management plan',
 'Lifestyle education regarding hypertension',
 'Self-care interventions (procedure)',
 'Musculoskeletal care',
 'Minor surgery care management (procedure)',
 'Asthma self management',
 'Allergic disorder monitoring',
 'Head injury rehabilitation',
 'Hyperlipidemia clinical management plan',
 'Skin condition care',
 'Burn care',
 'Care plan (record artifact)',
 'Cancer care plan',
 'Chronic obstructive pulmonary disease clinical management plan',
 'Mental health care plan',
 'Care Plan',
 'Overactivity/inattention behavior management',
 'Demential management',
 'Major surgery care management',
 'Terminal care',
 'Heart failure self management plan',
 'Inpatient care plan (record artifact)',
 'Spinal cord injury rehabilitation',
 'Dialysis care plan (record artifact)',
 'Psychiatry care plan',
 'Major depres

# Filter Pregnant Patients

In [9]:
careplans_df

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
1,07d9ddd8-dfa1-4e43-9bfe-39f63f4ace15,2011-05-13,2011-08-02,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
2,a3bb6e99-3b99-44b3-974c-e230b4511b5c,2011-12-31,2012-11-30,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,16300c56-a035-4126-a656-68c093da6dfc,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
3,9f5284b7-425a-486a-b36e-ab818c018f2f,2016-12-29,2017-01-05,034e9e3b-2def-4559-bb2a-7850888ae060,3b639086-5fbc-4720-8c31-e8c8c0f1d660,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
4,47ede16c-c216-4f81-a16b-0e858de9cdc3,2017-01-22,2017-02-12,10339b10-3cd1-4ac3-ac13-ec26728cb592,4ec8d55b-05fc-42a5-bfa3-1e233874a362,225358003,Wound care,284551006.0,Laceration of foot
...,...,...,...,...,...,...,...,...,...
3478,5998d466-9922-4a35-9277-7a902352ce48,1983-09-08,1983-09-29,fca3178e-fb68-41c3-8598-702d3ca68b96,9e9670e5-da7f-4111-98a6-21d5e0aadd86,91251008,Physical therapy procedure,70704007.0,Sprain of wrist
3479,3de3b598-ca98-4b98-947a-34e3e86630e6,1984-09-23,1984-11-22,fca3178e-fb68-41c3-8598-702d3ca68b96,8d7d70be-467d-430f-8497-25aa2c1276b8,385691007,Fracture care,263102004.0,Fracture subluxation of wrist
3480,6f02c458-e04f-429d-99ce-72626d87952d,1987-09-27,NaN,fca3178e-fb68-41c3-8598-702d3ca68b96,48ea35a8-37cd-41d2-86c1-7785b514c453,735984001,Heart failure self management plan,88805009.0,Chronic congestive heart failure (disorder)
3481,243de451-1e53-40fc-b4ae-e0ea730e4a0d,1990-03-15,NaN,fca3178e-fb68-41c3-8598-702d3ca68b96,aaee3543-3edb-40af-a276-3bb027cac0dc,736353004,Inpatient care plan (record artifact),NaN,NaN


In [10]:
len(careplans_df['ENCOUNTER'].unique())

3472

In [11]:
pregnant_patients = careplans_df[careplans_df['DESCRIPTION'] == 'Routine antenatal care']

In [12]:
len(pregnant_patients['PATIENT'].unique())

187

# Explore conditions data

In [13]:
conditions_df=df_dict['conditions_df']
conditions_df['START'] =pd.to_datetime(conditions_df['START'], infer_datetime_format=True)
conditions_df['STOP'] =pd.to_datetime(conditions_df['STOP'], infer_datetime_format=True)
conditions_df.rename(columns={'START':"DATE"}, inplace=True)
conditions_df

,DATE,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,NaT,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,Viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,Acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,Acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,Acute viral pharyngitis (disorder)
...,...,...,...,...,...,...
8371,1984-12-29,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,87cfa8a4-126e-4c58-a839-e8b725828d0f,64859006,Osteoporosis (disorder)
8372,1988-08-24,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,97c0226c-92b4-4e78-b81f-4225d8473ba3,40055000,Chronic sinusitis (disorder)
8373,2009-08-08,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,1c91f459-adf0-4fe6-abdb-d248a91b9d85,230690007,Stroke
8374,2010-11-18,2010-11-25,fc817953-cc8b-45db-9c85-7c0ced8fa90d,8aabb13d-f4ac-4037-95ab-00e0e48a1b34,444814009,Viral sinusitis (disorder)


In [14]:
conditions_df['DESCRIPTION'].value_counts()[:20]

Viral sinusitis (disorder)                  1248
Acute viral pharyngitis (disorder)           653
Acute bronchitis (disorder)                  563
Normal pregnancy                             516
Body mass index 30+ - obesity (finding)      449
Prediabetes                                  317
Hypertension                                 302
Anemia (disorder)                            300
Chronic sinusitis (disorder)                 236
Miscarriage in first trimester               221
Otitis media                                 196
Streptococcal sore throat (disorder)         157
Hyperlipidemia                               136
Sprain of ankle                              134
Polyp of colon                                79
Concussion with no loss of consciousness      77
Diabetes                                      76
Hypertriglyceridemia (disorder)               74
Metabolic syndrome X (disorder)               74
Acute bacterial sinusitis (disorder)          69
Name: DESCRIPTION, d

In [15]:
preg = conditions_df[conditions_df['DESCRIPTION'].isin(['Normal pregnancy',
'Miscarriage in first trimester',
'fetus with unknown complication',
'Tubal pregnancy',
'Antepartum eclampsia',
'Preeclampsia',
'Blighted ovum'
])]

In [16]:
len(set(preg['PATIENT'].unique()))

332

In [17]:
pregnant_patients_list = set(list(pregnant_patients['PATIENT'].unique()) + list(preg['PATIENT'].unique()))

In [18]:
len(pregnant_patients_list)

338

In [19]:
patients_df

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,AGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,NaT,999-76-6866,S99984236,X19277260X,Mr.,José Eduardo181,Gómez206,NaN,...,427 Balistreri Way Unit 19,Chicopee,Massachusetts,Hampden County,1013.0,42.228354,-72.562951,271227.08,1334.88,31.0
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,NaT,999-73-5361,S99962402,X88275464X,Mr.,Milo271,Feil794,NaN,...,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531,793946.01,3204.49,37.0
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,NaT,999-27-3385,S99972682,X73754411X,Mr.,Jayson808,Fadel536,NaN,...,1056 Harris Lane Suite 70,Chicopee,Massachusetts,Hampden County,1020.0,42.181642,-72.608842,574111.90,2606.40,28.0
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,NaT,999-85-4926,S99974448,X40915583X,Mrs.,Mariana775,Rutherford999,NaN,...,999 Kuhn Forge,Lowell,Massachusetts,Middlesex County,1851.0,42.636143,-71.343255,935630.30,8756.19,42.0
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,NaT,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,NaN,...,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,598763.07,3772.20,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,023a7d29-32b3-4db5-89c8-b88bd7582ec0,1922-02-14,NaT,999-60-9291,S99949872,X45944669X,Mrs.,Conchita9,Peres371,NaN,...,365 Gorczany Crossing Apt 64,Cambridge,Massachusetts,Middlesex County,2141.0,42.413390,-71.102083,1622314.87,32086.31,98.0
1167,1fc8dafd-90e5-44d6-9179-81156c0ea083,1977-08-08,NaT,999-29-3501,S99999143,X43299813X,Mr.,Laverne101,Hane680,NaN,...,177 Volkman Heights Suite 94,Beverly,Massachusetts,Essex County,1915.0,42.573337,-70.883400,979724.25,3130.52,43.0
1168,6d048a56-edb8-4f29-891d-7a84d75a8e78,1914-09-05,2006-05-26,999-60-2184,S99940416,X26292359X,Ms.,Kimber624,Volkman526,NaN,...,583 Price Rue,Norwood,Massachusetts,Norfolk County,NaN,42.211364,-71.213621,1560540.35,52391.24,92.0
1169,fca3178e-fb68-41c3-8598-702d3ca68b96,1914-09-05,1990-03-17,999-48-3257,S99949673,X25242364X,Mrs.,Shira43,Bruen238,NaN,...,537 Boehm Course,Norwood,Massachusetts,Norfolk County,2062.0,42.167146,-71.235461,1375833.47,13157.00,76.0


### Filter patients_df for only pregnant women

In [20]:
patients_df = patients_df[patients_df['Id'].isin(pregnant_patients_list)]

### Filter careplans_df for only pregnant women

In [21]:
careplans_df = careplans_df[careplans_df['PATIENT'].isin(pregnant_patients_list)]

### Filter conditions_df for only pregnant women

In [22]:
conditions_df = conditions_df[conditions_df['PATIENT'].isin(pregnant_patients_list)]

In [23]:
conditions_df

,DATE,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
23,2004-07-31,NaT,b58731cc-2d8b-4c2d-b327-4cab771af3ef,c855eddb-1301-493b-b1b1-7f5c3c5a934f,162864005,Body mass index 30+ - obesity (finding)
24,2014-05-05,2014-06-24,b58731cc-2d8b-4c2d-b327-4cab771af3ef,6fd85194-b6ad-48e9-8d2b-ac19318eacea,38822007,Cystitis
25,2014-07-08,2014-09-06,b58731cc-2d8b-4c2d-b327-4cab771af3ef,3a36836d-da25-4e73-808b-972b669b7e4e,65966004,Fracture of forearm
26,2016-03-20,NaT,b58731cc-2d8b-4c2d-b327-4cab771af3ef,d870177c-5819-4ca7-b802-545ef9ba5f11,196416002,Impacted molars
27,2018-05-26,2019-01-05,b58731cc-2d8b-4c2d-b327-4cab771af3ef,42cf4e99-a963-4fcc-a3c6-17bcc82876c1,72892002,Normal pregnancy
...,...,...,...,...,...,...
8371,1984-12-29,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,87cfa8a4-126e-4c58-a839-e8b725828d0f,64859006,Osteoporosis (disorder)
8372,1988-08-24,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,97c0226c-92b4-4e78-b81f-4225d8473ba3,40055000,Chronic sinusitis (disorder)
8373,2009-08-08,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,1c91f459-adf0-4fe6-abdb-d248a91b9d85,230690007,Stroke
8374,2010-11-18,2010-11-25,fc817953-cc8b-45db-9c85-7c0ced8fa90d,8aabb13d-f4ac-4037-95ab-00e0e48a1b34,444814009,Viral sinusitis (disorder)


In [24]:
# one-hot-encode conditions_df
conditions_df['DESCRIPTION'] = conditions_df['CODE'].astype(str) + '_' +conditions_df['DESCRIPTION']
conditions_df = conditions_df.drop(columns=['CODE'])
conditions_df = pd.get_dummies(conditions_df, columns = ['DESCRIPTION'])
conditions_df

,DATE,STOP,PATIENT,ENCOUNTER,DESCRIPTION_10509002_Acute bronchitis (disorder),DESCRIPTION_109838007_Overlapping malignant neoplasm of colon,DESCRIPTION_110030002_Concussion injury of brain,DESCRIPTION_124171000119105_Chronic intractable migraine without aura,DESCRIPTION_127013003_Diabetic renal disease (disorder),DESCRIPTION_128613002_Seizure disorder,...,DESCRIPTION_79586000_Tubal pregnancy,DESCRIPTION_80394007_Hyperglycemia (disorder),DESCRIPTION_82423001_Chronic pain,DESCRIPTION_83664006_Idiopathic atrophic hypothyroidism,DESCRIPTION_84757009_Epilepsy,DESCRIPTION_87433001_Pulmonary emphysema (disorder),DESCRIPTION_88805009_Chronic congestive heart failure (disorder),DESCRIPTION_90781000119102_Microalbuminuria due to type 2 diabetes mellitus (disorder),DESCRIPTION_94260004_Secondary malignant neoplasm of colon,DESCRIPTION_95417003_Primary fibromyalgia syndrome
23,2004-07-31,NaT,b58731cc-2d8b-4c2d-b327-4cab771af3ef,c855eddb-1301-493b-b1b1-7f5c3c5a934f,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,2014-05-05,2014-06-24,b58731cc-2d8b-4c2d-b327-4cab771af3ef,6fd85194-b6ad-48e9-8d2b-ac19318eacea,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,2014-07-08,2014-09-06,b58731cc-2d8b-4c2d-b327-4cab771af3ef,3a36836d-da25-4e73-808b-972b669b7e4e,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,2016-03-20,NaT,b58731cc-2d8b-4c2d-b327-4cab771af3ef,d870177c-5819-4ca7-b802-545ef9ba5f11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,2018-05-26,2019-01-05,b58731cc-2d8b-4c2d-b327-4cab771af3ef,42cf4e99-a963-4fcc-a3c6-17bcc82876c1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8371,1984-12-29,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,87cfa8a4-126e-4c58-a839-e8b725828d0f,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8372,1988-08-24,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,97c0226c-92b4-4e78-b81f-4225d8473ba3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8373,2009-08-08,NaT,fc817953-cc8b-45db-9c85-7c0ced8fa90d,1c91f459-adf0-4fe6-abdb-d248a91b9d85,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8374,2010-11-18,2010-11-25,fc817953-cc8b-45db-9c85-7c0ced8fa90d,8aabb13d-f4ac-4037-95ab-00e0e48a1b34,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
len(conditions_df['ENCOUNTER'].unique())

2859

# Explore observations data

### Filter observations_df for only pregnant women

In [249]:
observations_df = observations_df[observations_df['PATIENT'].isin(pregnant_patients_list)]

In [250]:
#join the units and description columns
observations_df = df_dict['observations_df']
observations_df['DATE'] =pd.to_datetime(observations_df['DATE'], infer_datetime_format=True)
observations_df['DESCRIPTION'] = observations_df['DESCRIPTION'].astype(str) + '_' +observations_df['UNITS']

In [251]:
observations_df['TYPE'].unique()

array(['numeric', 'text'], dtype=object)

In [252]:
#Divide df into numrrical and categotical values
cat_observations = observations_df[observations_df['TYPE']=='text'].drop(columns=['UNITS','TYPE'])
num_observations = observations_df[observations_df['TYPE']=='numeric'].drop(columns=['UNITS','TYPE'])

In [253]:
#cast data to float
num_observations = num_observations.astype({'VALUE':'float'})


In [254]:
num_observations['DESCRIPTION'].unique()

array(['Body Height_cm_cm_cm_cm_cm',
       'Pain severity - 0-10 verbal numeric rating [Score] - Reported_{score}_{score}_{score}_{score}_{score}',
       'Body Weight_kg_kg_kg_kg_kg',
       'Body Mass Index_kg/m2_kg/m2_kg/m2_kg/m2_kg/m2',
       'Diastolic Blood Pressure_mm[Hg]_mm[Hg]_mm[Hg]_mm[Hg]_mm[Hg]',
       'Systolic Blood Pressure_mm[Hg]_mm[Hg]_mm[Hg]_mm[Hg]_mm[Hg]',
       'Heart rate_/min_/min_/min_/min_/min',
       'Respiratory rate_/min_/min_/min_/min_/min',
       'Body temperature_Cel_Cel_Cel_Cel_Cel',
       'Body mass index (BMI) [Percentile] Per age and gender_%_%_%_%_%',
       'Leukocytes [#/volume] in Blood by Automated count_10*3/uL_10*3/uL_10*3/uL_10*3/uL_10*3/uL',
       'Erythrocytes [#/volume] in Blood by Automated count_10*6/uL_10*6/uL_10*6/uL_10*6/uL_10*6/uL',
       'Hemoglobin [Mass/volume] in Blood_g/dL_g/dL_g/dL_g/dL_g/dL',
       'Hematocrit [Volume Fraction] of Blood by Automated count_%_%_%_%_%',
       'MCV [Entitic volume] by Automated count_fL_f

In [255]:
#convert to wide table
num_observations = pd.pivot_table(num_observations, index=['DATE', 'PATIENT', 'ENCOUNTER'],columns='DESCRIPTION',values='VALUE')
num_observations_cols = num_observations.columns
num_observations = num_observations.reset_index()

In [256]:
num_observations = num_observations.groupby(['ENCOUNTER'])[num_observations_cols].mean()
num_observations

DESCRIPTION,Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma_U/L_U/L_U/L_U/L_U/L,Albumin [Mass/volume] in Serum or Plasma_g/dL_g/dL_g/dL_g/dL_g/dL,Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma_U/L_U/L_U/L_U/L_U/L,American house dust mite IgE Ab in Serum_kU/L_kU/L_kU/L_kU/L_kU/L,Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma_U/L_U/L_U/L_U/L_U/L,Bilirubin.total [Mass/volume] in Serum or Plasma_mg/dL_mg/dL_mg/dL_mg/dL_mg/dL,Bilirubin.total [Mass/volume] in Urine by Test strip_mg/dL_mg/dL_mg/dL_mg/dL_mg/dL,Body Height_cm_cm_cm_cm_cm,Body Mass Index_kg/m2_kg/m2_kg/m2_kg/m2_kg/m2,Body Weight_kg_kg_kg_kg_kg,...,Total score [MMSE]_(score)_(score)_(score)_(score)_(score),Triglycerides_mg/dL_mg/dL_mg/dL_mg/dL_mg/dL,Urea Nitrogen_mg/dL_mg/dL_mg/dL_mg/dL_mg/dL,WBC Auto (Bld) [#/Vol]_10*3/uL_10*3/uL_10*3/uL_10*3/uL_10*3/uL,Walnut IgE Ab in Serum_kU/L_kU/L_kU/L_kU/L_kU/L,Weight difference [Mass difference] --pre dialysis - post dialysis_kg_kg_kg_kg_kg,Weight-for-length Per age and sex_%_%_%_%_%,Wheat IgE Ab in Serum_kU/L_kU/L_kU/L_kU/L_kU/L,White oak IgE Ab in Serum_kU/L_kU/L_kU/L_kU/L_kU/L,pH of Urine by Test strip_pH_pH_pH_pH_pH
ENCOUNTER,,,,,,,,,,,,,,,,,,,,,
00055b87-0a03-4ca8-a69d-924bbb84033a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN
000c97b3-5832-4ecc-8a47-76c0197fffe4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174.7,27.8,84.9,...,NaN,120.4,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000c9d31-d256-4308-91a3-cf4181178ba2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.2,30.2,66.4,...,NaN,NaN,8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000d18f9-4f7f-41be-b874-7f5252550e62,48.0,3.6,97.5,NaN,26.5,1.1,1.3,NaN,NaN,NaN,...,NaN,NaN,16.5,NaN,NaN,NaN,NaN,NaN,NaN,5.0
0013356e-7526-47c9-bf67-35d5bd55dd91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176.8,27.3,85.4,...,NaN,188.2,19.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff773ea-5578-4535-b8a5-70441ab3662a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.0,30.4,79.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff797dd-5cd1-4c60-b3cd-22efd331e48d,23.2,5.3,80.9,NaN,23.4,1.2,NaN,NaN,NaN,NaN,...,NaN,NaN,17.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fff93412-1312-49f5-9ebd-8d4eda0d8de7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.5,27.2,71.8,...,NaN,145.4,9.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [258]:
# one-hot-encode cat_observations
cat_observations['VALUE'] = cat_observations['CODE'].astype(str) + '_' +cat_observations['VALUE']
cat_observations = cat_observations.drop(columns=['CODE','DESCRIPTION'])
cat_observations = pd.get_dummies(cat_observations, columns = ['VALUE'])
cat_observations

,DATE,PATIENT,ENCOUNTER,VALUE_10480-2_Positive (qualifier value),VALUE_20454-5_Urine protein test = + (finding),VALUE_20454-5_Urine protein test = ++ (finding),VALUE_20454-5_Urine protein test = +++ (finding),VALUE_21905-5_T1 category (finding),VALUE_21905-5_T2 category (finding),VALUE_21905-5_T3 category (finding),...,VALUE_85318-4_greater than 2.2,VALUE_85318-4_less than 1.8,VALUE_85319-2_Negative (qualifier value),VALUE_85319-2_Positive (qualifier value),VALUE_85337-4_Negative (qualifier value),VALUE_85337-4_Positive (qualifier value),VALUE_85339-0_Negative (qualifier value),VALUE_85339-0_Positive (qualifier value),VALUE_88040-1_Improving (qualifier value),VALUE_88040-1_Worsening (qualifier value)
20,2010-07-27 12:58:08+00:00,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37,2010-11-20 03:04:34+00:00,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49,2012-01-23 17:45:28+00:00,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69,2011-11-26 03:04:34+00:00,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1923c698-accd-4d70-ba09-e1938f6e96d1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,2011-07-28 15:02:18+00:00,1d604da9-9a81-4ba9-80c2-de3375d59b40,b85c339a-6076-43ed-b9d0-9cf013dec49d,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299611,2015-06-20 17:27:52+00:00,fc817953-cc8b-45db-9c85-7c0ced8fa90d,bd3bfc3c-4f52-4d9c-aa01-60a084b2ddcd,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299624,2016-06-25 17:27:52+00:00,fc817953-cc8b-45db-9c85-7c0ced8fa90d,59727285-d30f-44e2-bcda-eac21ce07269,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299644,2017-07-01 17:27:52+00:00,fc817953-cc8b-45db-9c85-7c0ced8fa90d,957b13d5-49fa-4518-8c25-6eaa45e6a7d1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299653,2018-07-07 17:27:52+00:00,fc817953-cc8b-45db-9c85-7c0ced8fa90d,5529a0d5-a24c-42b4-a32a-729e4d1f19ef,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
cat_observations_cols = set(cat_observations.columns) - set(['DATE', 'PATIENT', 'ENCOUNTER']) 


{'VALUE_10480-2_Positive (qualifier value)',
 'VALUE_20454-5_Urine protein test = + (finding)',
 'VALUE_20454-5_Urine protein test = ++ (finding)',
 'VALUE_20454-5_Urine protein test = +++ (finding)',
 'VALUE_21905-5_T1 category (finding)',
 'VALUE_21905-5_T2 category (finding)',
 'VALUE_21905-5_T3 category (finding)',
 'VALUE_21905-5_T4 category (finding)',
 'VALUE_21906-3_N0 category (finding)',
 'VALUE_21906-3_N1 category (finding)',
 'VALUE_21906-3_N2 category (finding)',
 'VALUE_21906-3_N3 category (finding)',
 'VALUE_21907-1_M0 category (finding)',
 'VALUE_21907-1_M1 category (finding)',
 'VALUE_21908-9_Stage 1 (qualifier value)',
 'VALUE_21908-9_Stage 1A (qualifier value)',
 'VALUE_21908-9_Stage 2 (qualifier value)',
 'VALUE_21908-9_Stage 2A (qualifier value)',
 'VALUE_21908-9_Stage 2B (qualifier value)',
 'VALUE_21908-9_Stage 3 (qualifier value)',
 'VALUE_21908-9_Stage 3A (qualifier value)',
 'VALUE_21908-9_Stage 3C (qualifier value)',
 'VALUE_21908-9_Stage 4 (qualifier value)'

In [264]:
cat_observations_cols = list(cat_observations_cols)
cat_observations = cat_observations.groupby(['ENCOUNTER'])[cat_observations_cols].mean()
cat_observations

,VALUE_21924-6_Negative (qualifier value),VALUE_72166-2_Former smoker,VALUE_85339-0_Positive (qualifier value),VALUE_21908-9_Stage 1 (qualifier value),VALUE_25428-4_Urine glucose test = ++ (finding),VALUE_28245-9_No signs/symptoms,VALUE_44667-4_None (qualifier value),VALUE_55277-8_not HIV positive,VALUE_5802-4_Urine nitrite negative (finding),VALUE_85337-4_Positive (qualifier value),...,VALUE_5794-3_Blood in urine (finding),VALUE_5794-3_Urine blood test = negative (finding),VALUE_72166-2_Never smoker,VALUE_20454-5_Urine protein test = +++ (finding),VALUE_5778-6_Reddish color urine,VALUE_69453-9_Non-small cell lung cancer (disorder),VALUE_21908-9_Stage 2A (qualifier value),VALUE_21908-9_Stage 3 (qualifier value),VALUE_63513-6_Yes,VALUE_85337-4_Negative (qualifier value)
ENCOUNTER,,,,,,,,,,,,,,,,,,,,,
000c97b3-5832-4ecc-8a47-76c0197fffe4,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
000c9d31-d256-4308-91a3-cf4181178ba2,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
000d18f9-4f7f-41be-b874-7f5252550e62,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.090909,0.0,...,0.0,0.090909,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0
0013356e-7526-47c9-bf67-35d5bd55dd91,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
001b0e21-67f3-4b39-947d-9caffbee4218,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffecb870-45f4-4fbb-8474-6a757723c703,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
fff6ffb6-ea12-4a67-b7cb-93a401048fab,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
fff773ea-5578-4535-b8a5-70441ab3662a,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [271]:
len(cat_observations.index.unique())

13431

In [272]:
#Join observations dfs

observations = num_observations.join(cat_observations)

In [27]:
observations

NameError: name 'observations' is not defined

In [155]:
pd.merge(num_observations, cat_observations, left_index=True, right_index=True)

SyntaxError: positional argument follows keyword argument (646110832.py, line 2)

In [156]:
len(cat_observations['PATIENT'].unique())

1171

In [157]:
num_observations['VALUE'].value_counts()

1.0       11382
0.0        6148
3.0        4752
2.0        4112
14.0       3748
          ...  
423.1         1
358.6         1
323.0         1
1661.3        1
931.7         1
Name: VALUE, Length: 4758, dtype: int64

In [ ]:
num_observations